<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


Before we get the data and start exploring it, let's import all the dependencies that we will need.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from bs4 import BeautifulSoup # library to parse HTML

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset


I selected the table element from the wikipedia page and saved it in a separate file. The code below finds all elements in the table and saves the information.

In [2]:
fname = "toronto.html" # file with <table>
with open(fname) as fopen:
    data = fopen.read()

parse = BeautifulSoup(data, 'html.parser')
table = []
for td in parse.findAll('td'):
    if td.span.text == 'Not assigned':
        continue
    else:
        postcode = td.p.text[:3]
        borough = td.span.text.split('(')[0]
        neighborhoods = ' '.join(td.span.text.split('(')[1:]).strip(')').replace(')',' ').replace(' / ',',')
        if 'Not assigned' in neighborhoods:
            neighborhoods = borough
        #else:
        #    neighborhoods = neighborhoods.split(',')
        table.append({'postcode':postcode, 'borough':borough, 'neighborhoods':neighborhoods})
    
    
table = pd.DataFrame(table)
## Clean names of boroughs
table['borough']=table['borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

table
        

,postcode,borough,neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern,Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill,Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Load and explore the data


Next, let's load the data.


In [3]:
fname = 'Geospatial_Coordinates.csv'
## Align both data frames so rows are consistent and can be merged
table = table.sort_values('postcode').reset_index().drop('index',axis=1)
coords_data = pd.read_csv(fname).sort_values('Postal Code').reset_index().drop('index',axis=1)
#print(table.tail())
#print(coords_data.tail())
table['latitude'] = coords_data['Latitude']
table['longitude'] = coords_data['Longitude']
table.shape

(103, 5)

Quickly examine the resulting dataframe.


In [4]:
table.head()

,postcode,borough,neighborhoods,latitude,longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


And make sure that the dataset has all 15 boroughs and 103 postal codes.


In [5]:
print('The dataframe has {} boroughs and {} postal codes.'.format(
        len(table['borough'].unique()),
        table.shape[0]
    )
)

The dataframe has 15 boroughs and 103 postal codes.


#### Use geopy library to get the latitude and longitude values of Toronto.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>toronto_explorer</em>, as shown below.


In [6]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with postal codes superimposed on top.


In [7]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, postcode in zip(table['latitude'], table['longitude'], table['borough'], table['postcode']):
    label = '{}, {}'.format(postcode, borough)
    color = 'yellow' # M1*
    if '2' in postcode: color = 'red' # M2
    elif '3' in postcode: color = 'blue' # M3
    elif '4' in postcode: color = 'green' # M4
    elif '5' in postcode: color = 'orange' # M5
    elif '6' in postcode: color = 'purple' # M6
    elif '7' in postcode: color = 'cyan' # M7
    elif '8' in postcode: color = 'gray' # M8
    elif '9' in postcode: color = 'white' # M9
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the post codes starting with M5. So let's slice the original dataframe and create a new dataframe of the Toronto data.


In [8]:
m5_data = table[table['postcode'].str.startswith('M5')].reset_index(drop=True)
m5_data.head()

,postcode,borough,neighborhoods,latitude,longitude
0,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


Let's get the geographical coordinates of Downtown Toronto.


In [9]:
address = 'Downtown Toronto, Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Downtown Toronto are 43.6563221, -79.3809161.


As we did with all of Toronto, let's visualize Downtown Toronto and the post codes in it.


In [10]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(m5_data['latitude'], m5_data['longitude'], m5_data['postcode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

Next, we are going to start utilizing the Foursquare API to explore the post codes and segment them.


#### Define Foursquare Credentials and Version


In [11]:
with open('.fs_client_id') as fopen:
    CLIENT_ID = fopen.read().strip() # your Foursquare ID
with open('.fs_client_secret') as fopen:
    CLIENT_SECRET = fopen.read().strip() # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:


#### Let's explore the first postcode in our dataframe.


Get the postcode's name.


In [12]:
m5_data.loc[0, 'postcode']

'M5A'

Get the postcode's latitude and longitude values.


In [13]:
postcode_latitude = m5_data.loc[0, 'latitude'] # neighborhood latitude value
postcode_longitude = m5_data.loc[0, 'longitude'] # neighborhood longitude value

postcode_name = m5_data.loc[0, 'postcode'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(postcode_name, 
                                                               postcode_latitude, 
                                                               postcode_longitude))

Latitude and longitude values of M5A are 43.6542599, -79.3606359.


#### Now, let's get the top 100 venues that are in M5A within a radius of 500 meters.


First, let's create the GET request URL. Name your URL **url**.


In [14]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, postcode_latitude, postcode_longitude, VERSION, radius, LIMIT) 
url


'https://api.foursquare.com/v2/venues/explore?client_id=45G3AFA1RMWI5ZBGA1CNMWIANV2IKBMQZ2055Q3FY4UFUWZ0&client_secret=CXGDLJOHSB3LKYZ2RIGELH5C2RS4L2CLRJHDWYLY4F1KO32S&ll=43.6542599,-79.3606359&v=20180605&radius=500&limit=100'

Double-click **here** for the solution.

<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 


Send the GET request and examine the resutls


In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '61088edef2332049b48270df'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 45,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Impact Kitchen,Restaurant,43.656369,-79.356980
4,Body Blitz Spa East,Spa,43.654735,-79.359874


And how many venues were returned by Foursquare?


In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

45 venues were returned by Foursquare.


## 2. Explore Postcodes starting with M5


#### Let's create a function to repeat the same process to all the postcodes that start with M5


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _manhattan_venues_.


In [20]:
# type your answer here
m5_venues = getNearbyVenues( m5_data['postcode'], m5_data['latitude'], m5_data['longitude'], radius = 1000)
m5_venues

M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X


,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,M5A,43.654260,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
5,M5A,43.654260,-79.360636,Corktown Common,43.655618,-79.356211,Park
6,M5A,43.654260,-79.360636,Distillery Sunday Market,43.650075,-79.361832,Farmers Market
7,M5A,43.654260,-79.360636,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
8,M5A,43.654260,-79.360636,Arvo,43.649963,-79.361442,Coffee Shop
9,M5A,43.654260,-79.360636,Rooster Coffee,43.651900,-79.365609,Coffee Shop


Double-click **here** for the solution.

<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 


#### Let's check the size of the resulting dataframe


In [21]:
print(m5_venues.shape)
m5_venues.head()

(1529, 7)


,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,M5A,43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


Let's check how many venues were returned for each neighborhood


In [22]:
m5_venues.groupby('Postcode').count()

,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
M5A,100,100,100,100,100,100
M5B,100,100,100,100,100,100
M5C,100,100,100,100,100,100
M5E,100,100,100,100,100,100
M5G,100,100,100,100,100,100
M5H,100,100,100,100,100,100
M5J,100,100,100,100,100,100
M5K,100,100,100,100,100,100
M5L,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues


In [23]:
print('There are {} uniques categories.'.format(len(m5_venues['Venue Category'].unique())))

There are 197 uniques categories.


## 3. Analyze Each Neighborhood


In [24]:
# one hot encoding
m5_onehot = pd.get_dummies(m5_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
m5_onehot['Postcode'] = m5_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [m5_onehot.columns[-1]] + list(m5_onehot.columns[:-1])
m5_onehot = m5_onehot[fixed_columns]

m5_onehot.head()

,Postcode,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Office,Optical Shop,Organic Grocery,Pakistani Restaurant,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

And let's examine the new dataframe size.


In [25]:
m5_onehot.shape

(1529, 198)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [26]:
m5_grouped = m5_onehot.groupby('Postcode').mean().reset_index()
m5_grouped

,Postcode,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Office,Optical Shop,Organic Grocery,Pakistani Restaurant,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M5A,0.000000,0.000000,0.00000,0.01,0.00,0.00,0.01,0.00,0.00,0.010000,0.01,0.01,0.00,0.00000,0.000000,0.040000,0.010000,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00000,0.00,0.03,0.01,0.00000,0.00,0.000000,0.01,0.00000,0.040000,0.00,0.00,0.00,0.00,0.00,0.01,0.000000,0.00,0.150000,0.00,0.00,0.00,0.00,0.00000,0.00,0.00,0.00000,0.00,0.010000,0.000000,0.00,0.01,0.03,0.01,0.000000,0.00,0.00,0.000000,0.00,0.00,0.01,0.01,0.00,0.01,0.000000,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.000000,0.000000,0.010000,0.00,0.01,0.00,0.00,0.00000,0.01000,0.000000,0.020000,0.010000,0.000000,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01000,0.00,0.00000,0.030000,0.010000,0.00,0.00,0.00,0.000000,0.01,0.00,0.00,0.000000,0.010000,0.00,0.010000,0.00,0.01,0.010000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.040000,0.00,0.02,0.000000,0.010000,0.010000,0.010000,0.000000,0.00,0.00,0.01,0.00,0.00,0.04000,0.00,0.00,0.030000,0.00,0.00,0.00,0.010000,0.000000,0.00,0.000000,0.00,0.01,0.00,0.010000,0.00,0.010000,0.01,0.00,0.00,0.00,0.00000,0.00,0.00,0.00,0.030000,0.00,0.00,0.00,0.00,0.00,0.01,0.02000,0.04,0.00000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00000,0.00,0.00,0.00000,0.01
1,M5B,0.000000,0.000000,0.01000,0.00,0.00,0.00,0.00,0.01,0.00,0.000000,0.00,0.00,

#### Let's confirm the new size


In [27]:
m5_grouped.shape

(18, 198)

#### Let's print each neighborhood along with the top 5 most common venues


In [28]:
num_top_venues = 5

for hood in m5_grouped['Postcode']:
    hoods = table[table['postcode'] == hood].values[0][2]
    print("----"+hoods+"----")
    temp = m5_grouped[m5_grouped['Postcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 3})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Regent Park,Harbourfront----
         venue  freq
0  Coffee Shop  0.15
1         Café  0.04
2      Theater  0.04
3       Bakery  0.04
4          Pub  0.04


----Garden District, Ryerson----
                venue  freq
0         Coffee Shop  0.09
1      Clothing Store  0.05
2           Gastropub  0.04
3  Italian Restaurant  0.04
4      Cosmetics Shop  0.03


----St. James Town----
            venue  freq
0            Café  0.08
1     Coffee Shop  0.08
2      Restaurant  0.06
3          Bakery  0.04
4  Clothing Store  0.03


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.06
2           Restaurant  0.06
3                Hotel  0.05
4  Japanese Restaurant  0.04


----Central Bay Street----
              venue  freq
0       Coffee Shop  0.09
1              Café  0.04
2  Sushi Restaurant  0.04
3              Park  0.03
4         Gastropub  0.03


----Richmond,Adelaide,King----
                 venue  freq
0          Coffee Shop  0.

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postcodes_venues_sorted = pd.DataFrame(columns=columns)
postcodes_venues_sorted['Postcode'] = m5_grouped['Postcode']

for ind in np.arange(m5_grouped.shape[0]):
    postcodes_venues_sorted.iloc[ind, 1:] = return_most_common_venues(m5_grouped.iloc[ind, :], num_top_venues)

postcodes_venues_sorted.head()


,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Coffee Shop,Pub,Café,Bakery,Park,Theater,Diner,Restaurant,Sushi Restaurant,Breakfast Spot
1,M5B,Coffee Shop,Clothing Store,Italian Restaurant,Gastropub,Cosmetics Shop,Japanese Restaurant,Restaurant,Diner,Middle Eastern Restaurant,Burrito Place
2,M5C,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Seafood Restaurant,Hotel,Clothing Store,Japanese Restaurant,Gastropub
3,M5E,Coffee Shop,Restaurant,Café,Hotel,Japanese Restaurant,Park,Bakery,Beer Bar,Cocktail Bar,Gym
4,M5G,Coffee Shop,Sushi Restaurant,Café,Park,Gastropub,Clothing Store,Yoga Studio,Ramen Restaurant,Mexican Restaurant,Japanese Restaurant


## 4. Cluster Postcodes


Run _k_-means to cluster the postcodes into 3 clusters.


In [31]:
# set number of clusters
kclusters = 3

m5_grouped_clustering = m5_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(m5_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 2, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each postcode.


In [32]:
# add clustering labels
postcodes_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

m5_merged = m5_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
m5_merged = m5_merged.join(postcodes_venues_sorted.set_index('Postcode'), on='postcode')

m5_merged.head() # check the last columns!

,postcode,borough,neighborhoods,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Café,Bakery,Park,Theater,Diner,Restaurant,Sushi Restaurant,Breakfast Spot
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Italian Restaurant,Gastropub,Cosmetics Shop,Japanese Restaurant,Restaurant,Diner,Middle Eastern Restaurant,Burrito Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Seafood Restaurant,Hotel,Clothing Store,Japanese Restaurant,Gastropub
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Restaurant,Café,Hotel,Japanese Restaurant,Park,Bakery,Beer Bar,Cocktail Bar,Gym
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Sushi Restaurant,Café,Park,Gastropub,Clothing Store,Yoga Studio,Ramen Restaurant,Mexican Restaurant,Japanese Restaurant


Finally, let's visualize the resulting clusters


In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(m5_merged['latitude'], m5_merged['longitude'], m5_merged['postcode'], m5_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 0


In [36]:
m5_merged.loc[m5_merged['Cluster Labels'] == 0, m5_merged.columns[[1] + list(range(5, m5_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,North York,0,Coffee Shop,Italian Restaurant,Bank,Fast Food Restaurant,Sandwich Place,Bridal Shop,Indian Restaurant,Intersection,Skating Rink,Boutique
10,Central Toronto,0,Sushi Restaurant,Italian Restaurant,Spa,Pharmacy,Coffee Shop,Café,Bank,Playground,Skating Rink,Dry Cleaner
11,Central Toronto,0,Park,Café,Bank,Coffee Shop,Sushi Restaurant,Liquor Store,Japanese Restaurant,Italian Restaurant,Playground,Pharmacy


#### Cluster 1


In [37]:
m5_merged.loc[m5_merged['Cluster Labels'] == 1, m5_merged.columns[[1] + list(range(5, m5_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Pub,Café,Bakery,Park,Theater,Diner,Restaurant,Sushi Restaurant,Breakfast Spot
1,Downtown Toronto,1,Coffee Shop,Clothing Store,Italian Restaurant,Gastropub,Cosmetics Shop,Japanese Restaurant,Restaurant,Diner,Middle Eastern Restaurant,Burrito Place
2,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Seafood Restaurant,Hotel,Clothing Store,Japanese Restaurant,Gastropub
3,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Hotel,Japanese Restaurant,Park,Bakery,Beer Bar,Cocktail Bar,Gym
4,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Café,Park,Gastropub,Clothing Store,Yoga Studio,Ramen Restaurant,Mexican Restaurant,Japanese Restaurant
5,Downtown Toronto,1,Coffee Shop,Café,Hotel,Theater,Restaurant,Japanese Restaurant,Clothing Store,Italian Restaurant,Pub,Breakfast Spot
6,Downtown Toronto,1,Café,Coffee Shop,Hotel,Park,Japanese Restaurant,Restaurant,Gym,Brewery,Scenic Lookout,Theater
7,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Theater,Bakery,Park,Monument / Landmark,Lounge
8,Downtown Toronto,1,Coffee Shop,Hotel,Restaurant,Café,Bakery,Seafood Restaurant,Cosmetics Shop,Gastropub,Beer Bar,Japanese Restaurant
12,Central Toronto,1,Italian Restaurant,Café,Coffee Shop,Gym,Restaurant,Mexican Restaurant,Grocery Store,Vegetarian / Vegan Restaurant,Bakery,Pub


#### Cluster 2


In [38]:
m5_merged.loc[m5_merged['Cluster Labels'] == 2, m5_merged.columns[[1] + list(range(5, m5_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Downtown Toronto,2,Coffee Shop,Café,Harbor / Marina,Airport,Track,Sculpture Garden,Scenic Lookout,Dog Run,Garden,Dance Studio
